In [96]:
import pandas as pd
import pulp
import math

In [67]:
demand_df = pd.read_csv("data/demand.csv")
vehicles_df = pd.read_csv("top_ranked_combined.csv")
carbon_limits_df = pd.read_csv("data/carbon_emissions.csv")
current_fleet = pd.read_csv("current_fleet.csv")

In [68]:
demand_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Year         256 non-null    int64 
 1   Size         256 non-null    object
 2   Distance     256 non-null    object
 3   Demand (km)  256 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 8.1+ KB


In [69]:
vehicles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Operating Year           256 non-null    int64  
 1   Size                     256 non-null    object 
 2   Distance_demand          256 non-null    object 
 3   Demand (km)              256 non-null    int64  
 4   ID                       256 non-null    object 
 5   Vehicle                  256 non-null    object 
 6   Available Year           256 non-null    int64  
 7   Cost ($)                 256 non-null    int64  
 8   Yearly range (km)        256 non-null    int64  
 9   Distance_vehicle         256 non-null    object 
 10  Fuel                     256 non-null    object 
 11  carbon_emissions_per_km  256 non-null    float64
 12  insurance_cost           256 non-null    float64
 13  maintenance_cost         256 non-null    float64
 14  fuel_costs_per_km        2

In [70]:
current_fleet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Operating Year           0 non-null      object
 1   Size                     0 non-null      object
 2   Distance_demand          0 non-null      object
 3   Demand (km)              0 non-null      object
 4   ID                       0 non-null      object
 5   Vehicle                  0 non-null      object
 6   Available Year           0 non-null      object
 7   Cost ($)                 0 non-null      object
 8   Yearly range (km)        0 non-null      object
 9   Distance_vehicle         0 non-null      object
 10  Fuel                     0 non-null      object
 11  carbon_emissions_per_km  0 non-null      object
 12  insurance_cost           0 non-null      object
 13  maintenance_cost         0 non-null      object
 14  fuel_costs_per_km        0 non-null      object
 15  T

In [71]:
carbon_limits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   Year                    16 non-null     int64
 1   Carbon emission CO2/kg  16 non-null     int64
dtypes: int64(2)
memory usage: 388.0 bytes


In [72]:
prob = pulp.LpProblem("Fleet_Optimization", pulp.LpMinimize)

In [73]:
vehicles_df

,Operating Year,Size,Distance_demand,Demand (km),ID,Vehicle,Available Year,Cost ($),Yearly range (km),Distance_vehicle,Fuel,carbon_emissions_per_km,insurance_cost,maintenance_cost,fuel_costs_per_km,Topsis_Score,Rank
0,2023,S1,D1,869181,LNG_S1_2023,LNG,2023,100000,102000,D4,BioLNG,0.062503,5000.00,1000.00,0.179663,0.896626,1
1,2023,S1,D2,2597094,Diesel_S1_2023,Diesel,2023,85000,102000,D4,HVO,0.108748,4250.00,850.00,0.404620,0.925539,1
2,2023,S1,D3,3292011,Diesel_S1_2023,Diesel,2023,85000,102000,D4,HVO,0.108748,4250.00,850.00,0.404620,0.925539,1
3,2023,S1,D4,414315,Diesel_S1_2023,Diesel,2023,85000,102000,D4,HVO,0.108748,4250.00,850.00,0.404620,0.925539,1
4,2023,S2,D1,995694,LNG_S2_2023,LNG,2023,133000,106000,D4,BioLNG,0.063269,6650.00,1330.00,0.181867,0.884463,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,2038,S3,D4,306910,BEV_S3_2038,BEV,2038,215849,73000,D4,Electricity,0.000000,10792.45,2158.49,0.089311,0.928088,1
252,2038,S4,D1,23437,BEV_S4_2038,BEV,2038,276287,118000,D4,Electricity,0.000000,13814.35,2762.87,0.088993,0.927744,1
253,2038,S4,D2,1148555,BEV_S4_2038,BEV,2038,276287,118000,D4,Electricity,0.000000,13814.35,2762.87,0.088993,0.927744,1
254,2038,S4,D3,161386,BEV_S4_2038,BEV,2038,276287,118000,D4,Electricity,0.000000,13814.35,2762.87,0.088993,0.927744,1


In [74]:
row = vehicles_df[(vehicles_df['Available Year'] == 2023) & 
                  (vehicles_df['Distance_demand'] == 'D1') & 
                  (vehicles_df['Size'] == 'S1')]


In [75]:
row

,Operating Year,Size,Distance_demand,Demand (km),ID,Vehicle,Available Year,Cost ($),Yearly range (km),Distance_vehicle,Fuel,carbon_emissions_per_km,insurance_cost,maintenance_cost,fuel_costs_per_km,Topsis_Score,Rank
0,2023,S1,D1,869181,LNG_S1_2023,LNG,2023,100000,102000,D4,BioLNG,0.062503,5000.0,1000.0,0.179663,0.896626,1


In [ ]:
count = 0
for year in range(2023, 2039):
    unique_demand_combination_for_year = demand_df.groupby(['Size', 'Distance'])
    carbon_limit = carbon_limits_df[carbon_limits_df['Year'] == year]['Carbon emission CO2/kg'].values[0]
    
    for (size, distance), group in unique_demand_combination_for_year:
        row = demand_df[(demand_df['Size'] == size) & 
                   (demand_df['Distance'] == distance) & 
                   (demand_df['Year'] == year)]
        
        demand = row['Demand (km)'].values[0] if not row.empty else None
        available_vehicles = []
        row = vehicles_df[(vehicles_df['Available Year'] == year) & 
                        (vehicles_df['Distance_demand'] == distance) & 
                        (vehicles_df['Size'] == size)]
        if not row.empty:
            available_vehicles.append(row['ID'].values[0])

        if len(current_fleet) > 0:
            row =  current_fleet[(current_fleet['Available Year'] == year) & 
                        (current_fleet['Distance_demand'] == distance) & 
                        (current_fleet['Size'] == size)]
            if not row.empty:
                available_vehicles.append(row['ID'].values[0])
        
        prob = pulp.LpProblem(f"Fleet_Optimization_{year}_{size}_{distance}", pulp.LpMinimize)
        
        for id in available_vehicles:
            row = vehicles_df[(vehicles_df['ID'] == id)]
            yearly_range = row['Yearly range (km)'].values[0] if not row.empty else None
            ub = math.ceil(demand/yearly_range)
            vehicle_vars = {id: pulp.LpVariable(f"num_vehicles_{id}", lowBound=0, upBound= ub, cat='Integer')}
            
                
        # Continuing the optimization logic
        # print(size, distance)
        # print(ub)

        # Objective: Minimize total cost
        for id in available_vehicles:
            prob += pulp.lpSum(vehicle_vars[id] * (row['Cost ($)'].values[0] + 
                                              row['insurance_cost'].values[0] + 
                                              row['maintenance_cost'].values[0] + 
                                              row['fuel_costs_per_km'].values[0] * 
                                              row['Yearly range (km)'].values[0]))
        
            # Constraint: Meet the demand
            prob += pulp.lpSum(vehicle_vars[id] * row['Yearly range (km)'].values[0]) >= demand
            
            # Constraint: Stay within carbon emission limits
            prob += pulp.lpSum(vehicle_vars[id] * row['carbon_emissions_per_km'].values[0] * row['Yearly range (km)'].values[0]) <= carbon_limit
            
            
            
            
        # Solve the optimization problem
        prob.solve()
        # print(vehicle_vars[id].varValue)
        if vehicle_vars[id].varValue > 0:
                fleet_decision = {
                    'Operating Year': year,
                    'Size': size,
                    'Distance': distance,
                    'Demand (km)': demand,
                    'ID': id,
                    'Num_Vehicles': int(vehicle_vars[id].varValue),
                    'Type': 'Buy' if year == 2023 else 'Use',
                    'Fuel': row['Fuel'].values[0]
                }
                print(fleet_decision)

print("Fleet optimization complete. Check 'optimized_fleet_plan.csv' for the plan.")

{'Operating Year': 2023, 'Size': 'S1', 'Distance': 'D1', 'Demand (km)': np.int64(869181), 'ID': 'LNG_S1_2023', 'Num_Vehicles': 9, 'Type': 'Buy', 'Fuel': 'BioLNG'}
{'Operating Year': 2023, 'Size': 'S1', 'Distance': 'D2', 'Demand (km)': np.int64(2597094), 'ID': 'Diesel_S1_2023', 'Num_Vehicles': 26, 'Type': 'Buy', 'Fuel': 'HVO'}
{'Operating Year': 2023, 'Size': 'S1', 'Distance': 'D3', 'Demand (km)': np.int64(3292011), 'ID': 'Diesel_S1_2023', 'Num_Vehicles': 33, 'Type': 'Buy', 'Fuel': 'HVO'}
{'Operating Year': 2023, 'Size': 'S1', 'Distance': 'D4', 'Demand (km)': np.int64(414315), 'ID': 'Diesel_S1_2023', 'Num_Vehicles': 5, 'Type': 'Buy', 'Fuel': 'HVO'}
{'Operating Year': 2023, 'Size': 'S2', 'Distance': 'D1', 'Demand (km)': np.int64(995694), 'ID': 'LNG_S2_2023', 'Num_Vehicles': 10, 'Type': 'Buy', 'Fuel': 'BioLNG'}
{'Operating Year': 2023, 'Size': 'S2', 'Distance': 'D2', 'Demand (km)': np.int64(1383196), 'ID': 'Diesel_S2_2023', 'Num_Vehicles': 14, 'Type': 'Buy', 'Fuel': 'HVO'}
{'Operating Yea